In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import gc, pickle

In [2]:
train_data = pd.read_csv("./csv/train_data.csv", index_col='id')
test_data = pd.read_csv("./csv/test_data.csv", index_col='id')
print(train_data.shape,test_data.shape)

(8878, 2) (24838, 1)


In [3]:
X = train_data['contents']
y = train_data['y']

In [12]:
#scikit-learnライブラリをimport
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import confusion_matrix, f1_score
# 回帰モデルのインスタンス化
#tfidf = TfidfVectorizer()
#tfidf.fit(X)
#vec_X = tfidf.transform(X)

countvec = CountVectorizer()
countvec.fit(X)
vec_X = countvec.transform(X)


In [46]:
count = 100

for a in np.linspace(0.001,0.04,10):
    rate = 0
    pred_rate = 0
    for i in range(count):
        X_train, X_test, y_train, y_test = train_test_split(
            vec_X, y, test_size=0.25, stratify=y)
        model = MultinomialNB(alpha=a)
        result = model.fit(X_train, y_train)
        acc = f1_score(y_train,model.predict(X_train))
        pred_acc = f1_score(y_test,model.predict(X_test))
        if pred_acc > 0.80: 
            pickle.dump(model,open('nb_model.pickle','wb'))
        #print(acc,' / ',pred_acc)
        rate += acc
        pred_rate += pred_acc
    print('alpha = ',a,' Average Rate:',rate/count, 'Pred Ave :', pred_rate/ count)

alpha =  0.001  Average Rate: 0.998639338823032 Pred Ave : 0.7831099941631714
alpha =  0.005333333333333333  Average Rate: 0.99342124615481 Pred Ave : 0.8190139904730387
alpha =  0.009666666666666667  Average Rate: 0.9908881147117353 Pred Ave : 0.8181771551209072
alpha =  0.013999999999999999  Average Rate: 0.9867346408234462 Pred Ave : 0.8183735940336994
alpha =  0.018333333333333333  Average Rate: 0.984880126415679 Pred Ave : 0.8154400262502227
alpha =  0.02266666666666667  Average Rate: 0.9831094899615052 Pred Ave : 0.8109258777074078
alpha =  0.027  Average Rate: 0.9800390063693252 Pred Ave : 0.8033381659990624
alpha =  0.03133333333333333  Average Rate: 0.9759846775934412 Pred Ave : 0.7991050840669561
alpha =  0.035666666666666666  Average Rate: 0.968707087860865 Pred Ave : 0.7889191794714873
alpha =  0.04  Average Rate: 0.9639911370144401 Pred Ave : 0.7878536368278906


In [47]:
print(rate/count)

0.9639911370144401


In [48]:
model = pickle.load(open('nb_model.pickle','rb'))

In [49]:
print(confusion_matrix(y_train, model.predict(X_train)))

[[6513   17]
 [   0  128]]


In [50]:
print(f1_score(y_train, model.predict(X_train))) #ほとんど検出できず

0.9377289377289377


In [51]:
print(confusion_matrix(y_test, model.predict(X_test)))

[[2172    5]
 [   0   43]]


In [52]:
print(f1_score(y_test, model.predict(X_test))) 

0.945054945054945


In [53]:
vec_test = countvec.transform(test_data['contents'])

In [54]:
#テスト結果の出力
test_predicted = model.predict(vec_test)
submit_df = pd.DataFrame({'y': test_predicted})
submit_df.index.name = 'id'
submit_df.index += 1
submit_df.to_csv('./sub/NB_submission.csv')